In [1]:
import pandas as pd

In [71]:
df1 = pd.read_excel("Training.xlsx")
df2 = pd.read_excel("Validation.xlsx")

In [72]:
df1.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)
df2.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)

In [79]:
df1.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df1.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)

df2.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df2.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)


In [80]:
df_train = pd.concat([
    df1[['Emotion', 'sentence']],
    df1[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df1[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)

df_valid = pd.concat([
    df2[['Emotion', 'sentence']],
    df2[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df2[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)


In [84]:
df_train = df_train.dropna()
df_valid = df_valid.dropna()

In [88]:
df_train.to_csv("training_data.csv", index=False)
df_valid.to_csv("validation_data.csv", index=False)

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("training_data.csv")
df_valid = pd.read_csv("validation_data.csv")

In [3]:
import re
from tqdm import tqdm

def removing_non_korean(df):
    for idx, row in tqdm(df.iterrows(), desc='removing_non_korean', total=len(df)):
        new_doc = re.sub('[^가-힣]', ' ', row['sentence']).strip()
        df.loc[idx, 'sentence'] = new_doc
    return df

train = removing_non_korean(df_train)
test = removing_non_korean(df_valid)

removing_non_korean: 100%|██████████| 17968/17968 [00:01<00:00, 10546.64it/s]


In [5]:
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [6]:
okt = Okt()

temp = []
for i in train['sentence']:
    temp.append(okt.morphs(i))
    
temp = pd.Series(temp)
train['token'] = temp

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145955 entries, 0 to 145954
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Emotion   145955 non-null  object
 1   sentence  145955 non-null  object
 2   token     145955 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [9]:
df_train.to_csv("training_data-token.csv", index=False)